In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext
from pyspark.sql.dataframe import DataFrame


from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import from_json
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, PCA
from pyspark.ml.classification import LogisticRegression
from pyspark import SparkContext
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch
import pyspark


In [2]:
import os
os.environ

environ{'HOSTNAME': '2fbfb92b6cc1',
        'JAVA_HOME': '/usr/local/openjdk-8',
        'PWD': '/opt/spark-2.4.8-bin-hadoop2.7',
        'HOME': '/root',
        'LANG': 'C.UTF-8',
        'SPARK_ACTION': 'pyspark',
        'PYSPARK_PYTHON': '/usr/bin/python3',
        'PYTHONPATH': '/opt/spark/python/:/opt/spark/python/lib/py4j-0.10.7-src.zip:/opt/spark/python/lib/pyspark.zip',
        'SPARK_DIR': '/opt/spark',
        'HADOOP_VERSION': '2.7',
        'SPARK_VERSION': '2.4.8',
        'SHLVL': '1',
        'SPARK_HOME': '/opt/spark',
        'PATH': '/opt/spark/bin:/usr/local/openjdk-8/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'JAVA_VERSION': '8u332',
        '_': '/usr/local/bin/jupyter',
        'PYDEVD_USE_FRAME_EVAL': 'NO',
        'JPY_PARENT_PID': '32',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://matplotlib_inline.backend_inline'}

In [3]:
!ls /usr/lib/

apt	     dpkg   gnupg2   mime	 ssl	     udev
bfd-plugins  gcc    gold-ld  os-release  systemd     valgrind
compat-ld    gnupg  locale   sasl2	 tmpfiles.d  x86_64-linux-gnu


In [4]:
spark = SparkSession.builder.master("local[2]") \
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark-2.4.8-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.11 added as a dependency
org.elasticsearch#elasticsearch-spark-20_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8637c5ec-4819-4e57-b141-d2e50e6c5734;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.11;2.4.7 in central
	found org.apache.kafka#kafka-clients;2.0.0 in central
	found org.lz4#lz4-java;1.4.0 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.16 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.elasticsearch#elasticsearch-spark-20_2.11;7.17.4 in central
	found org.scala-lang#scala-reflect;2.11.12 in central
	found commons-logging#commons-logging;1.1.1 in central
	found ja

In [6]:
es_mapping = {
    "mappings": {
        "properties": {
            "Nome": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                            "ignore_above": 256
                    }
                }
            },
            "vector": {
                    "type": "dense_vector",
                    "dims": 200
            },
            "pca": {
                "type": "point"
            },
        }
    }
}

es = Elasticsearch(hosts="http://10.0.100.51:9200") 
response = es.indices.create(
    index="vectorsteps",
    body=es_mapping,
    ignore=400 # ignore 400 already exists code
)
if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


In [7]:
schema = StructType([ \
    StructField("Link",StringType(),True), \
    StructField("Steps",StringType(),True), \
    StructField("Ingredienti",ArrayType(StructType([
         StructField('Nome', StringType(), True),
         StructField('Peso', StringType(), True),
         StructField('Note', StringType(), True)
         ])),True), \
    StructField("Nome", StringType(), True)
  ])

In [8]:
tr = spark.read\
  .format("kafka")\
  .option("subscribe", "CleanedRecipes")\
  .option("kafka.bootstrap.servers", "10.0.100.23:9092")\
  .option("kafka.request.timeout.ms", "60000")\
  .option("kafka.session.timeout.ms", "60000")\
  .option("failOnDataLoss", "true")\
  .option("startingOffsets", "earliest") \
  .option("partition", 1) \
  .option("fetch.message.max.bytes","52428800")\
  .option("kafka.max.poll.records", "1000")\
  .load()\
  .limit(1000)\

tr = tr.selectExpr("CAST(value AS STRING)")
tr = tr.select(from_json("value", schema).alias("data")) \
    .select("data.*")    
tr


DataFrame[Link: string, Steps: string, Ingredienti: array<struct<Nome:string,Peso:string,Note:string>>, Nome: string]

In [9]:
tr.count()

1000

In [10]:
stage_1 = RegexTokenizer(inputCol= 'Steps' , outputCol= 'steptokens', pattern= '\\W')
# define stage 2: remove the stop words
stage_2 = StopWordsRemover(inputCol= 'steptokens', outputCol= 'filtered_words')
# define stage 3: create a word vector of the size 100
stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize= 200)
# setup the pipeline

pca = PCA(k=2, inputCol="vector")
pca.setOutputCol("pca")

pipeline = Pipeline(stages= [stage_1, stage_2, stage_3,pca])

In [11]:
pipelineFit = pipeline.fit(tr)


In [12]:
ds = pipelineFit.transform(tr).select("Nome","pca")

In [13]:
ds.show()

+--------------------+--------------------+
|                Nome|                 pca|
+--------------------+--------------------+
|            Tiramisù|[-0.1007319794162...|
|Pancake allo scir...|[-0.0292037807742...|
|Spaghetti alla Ca...|[0.11258745261459...|
|Crepe dolci e sal...|[-0.1155195015620...|
|  Pasta per la pizza|[-0.1334991362682...|
| New York Cheesecake|[-0.0244016179456...|
|Spaghetti all'Ama...|[0.18186021881662...|
|      Torta tenerina|[-0.0751187076385...|
|Tortino di ciocco...|[-0.0554144511031...|
|            Brownies|[7.25502825981291...|
|        Banana bread|[-0.0027489154839...|
|         Besciamella|[0.00352168886721...|
|Spaghetti alle vo...|[0.14287876918852...|
|Spaghetti Cacio e...|[0.04638765563924...|
|Parmigiana di mel...|[0.10785743562569...|
|Muffin con gocce ...|[-0.0610205523530...|
|Lasagne alla Bolo...|[-0.0041416940590...|
|Crostata di frago...|[-0.0190113538053...|
|       Torta di mele|[0.03998592626912...|
|        Chiffon cake|[-0.092910

In [14]:
df = spark.readStream\
  .format("kafka")\
  .option("subscribe", "CleanedRecipes")\
  .option("kafka.bootstrap.servers", "10.0.100.23:9092")\
  .option("kafka.request.timeout.ms", "60000")\
  .option("kafka.session.timeout.ms", "60000")\
  .option("failOnDataLoss", "true")\
  .option("startingOffsets", "earliest") \
  .option("partition", 1) \
  .option("fetch.message.max.bytes","52428800")\
  .load()\
  .selectExpr("CAST(value AS STRING)")\
  .select(from_json("value", schema).alias("data")) \
  .select("data.*")    

df = pipelineFit.transform(df).select("Nome","vector","pca")

In [15]:
import pyspark.sql.functions as F
import pyspark.sql.types as T


to_array = F.udf(lambda v: v.toArray().tolist(), T.ArrayType(T.FloatType()))
df = df.withColumn('pca', to_array('pca'))
df = df.withColumn('vector', to_array('vector'))
df

DataFrame[Nome: string, vector: array<float>, pca: array<float>]

In [ ]:
df.writeStream \
    .outputMode("append") \
    .format("org.elasticsearch.spark.sql")\
    .option("es.nodes","10.0.100.51")\
    .option("checkpointLocation", "/save/location") \
    .start("vectorsteps") \
    .awaitTermination()
 
spark.streams.awaitAnyTermination()